## Imports

In [ ]:
import pandas as pd
from IPython.display import display
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score,r2_score
from sklearn.model_selection import train_test_split, KFold
# Plotting

from matplotlib import pyplot
import seaborn as sns 
import plotly
import plotly.offline as py
from plotly.offline import iplot
import plotly.graph_objs as go
import cufflinks as cf

%matplotlib inline
import requests
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
py.init_notebook_mode(connected=True)
cf.go_offline()
from IPython.display import display
#pd.options.display.max_columns = None
#pd.options.display.max_rows = Non

!pip install wtk

import logging
logging.basicConfig(level=logging.DEBUG)
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn import svm
from sklearn.model_selection import cross_val_score
from wtk.utilities import get_ucr_dataset, krein_svm_grid_search
from wtk import transform_to_dist_matrix, get_kernel_matrix
from sklearn.metrics import r2_score, log_loss

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
shot_chart_lebron = pd.read_csv('/kaggle/input/lebron_shooting_data.csv')

In [ ]:
shot_chart_lebron

### EDA

#### Game event ID

In [ ]:
print(shot_chart_lebron["GAME_EVENT_ID"].value_counts())
len(shot_chart_lebron["GAME_EVENT_ID"].unique())

#### Event Type

In [ ]:
print(shot_chart_lebron["EVENT_TYPE"].value_counts())
len(shot_chart_lebron["EVENT_TYPE"].unique())

In [ ]:
temp = shot_chart_lebron["EVENT_TYPE"].value_counts()
temp = (temp / temp.sum())*100
temp.iplot(kind='bar', labels='labels', values='values',
                                                     colors ='green', title='lebron shot performance')

#### Action type

In [ ]:
print(shot_chart_lebron["ACTION_TYPE"].value_counts())
print(len(shot_chart_lebron["ACTION_TYPE"].unique()))

In [ ]:
temp = shot_chart_lebron["ACTION_TYPE"].value_counts()
temp = (temp / temp.sum())*100
temp.iplot(kind='bar', labels='labels', values='values',
                                                     colors ='green', title='lebron action types')

As you can see above, for the Action type column, we have 45 different action types and their frequencies

#### Shot type

In [ ]:
print(shot_chart_lebron["SHOT_TYPE"].value_counts())
print(len(shot_chart_lebron["SHOT_TYPE"].unique()))

In [ ]:
temp = shot_chart_lebron["SHOT_TYPE"].value_counts()
temp = (temp / temp.sum())*100
temp.iplot(kind='bar', labels='labels', values='values',
                                                     colors ='green', title='lebron shot types')

#### SHOT ZONE BASIC

In [ ]:
print(shot_chart_lebron["SHOT_ZONE_BASIC"].value_counts())
print(len(shot_chart_lebron["SHOT_ZONE_BASIC"].unique()))

In [ ]:
temp = shot_chart_lebron["SHOT_ZONE_BASIC"].value_counts()
temp = (temp / temp.sum())*100
temp.iplot(kind='bar', labels='labels', values='values',
                                                     colors ='green', title='lebron shot zone basics')

#### SHOT_ZONE_AREA

In [ ]:
print(shot_chart_lebron["SHOT_ZONE_AREA"].value_counts())
print(len(shot_chart_lebron["SHOT_ZONE_AREA"].unique()))

In [ ]:
temp = shot_chart_lebron["SHOT_ZONE_AREA"].value_counts()
temp = (temp / temp.sum())*100
temp.iplot(kind='bar', labels='labels', values='values',
                                                     colors ='green', title='lebron shot zone area')

#### SHOT_DISTANCE

In [ ]:
print(shot_chart_lebron["SHOT_DISTANCE"].value_counts())
print(len(shot_chart_lebron["SHOT_DISTANCE"].unique()))

In [ ]:
temp = shot_chart_lebron["SHOT_DISTANCE"].value_counts()
temp = (temp / temp.sum())*100
temp.iplot(kind='bar', labels='labels', values='values',
                                                     colors ='green', title='lebron shot distance')

In [ ]:
from matplotlib.patches import Circle, Rectangle, Arc

def draw_court(ax=None, color='black', lw=2, outer_lines=False):
    # If an axes object isn't provided to plot onto, just get current one
    if ax is None:
        ax = plt.gca()

    # Create the various parts of an NBA basketball court

    # Create the basketball hoop
    # Diameter of a hoop is 18" so it has a radius of 9", which is a value
    # 7.5 in our coordinate system
    hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=color, fill=False)

    # Create backboard
    backboard = Rectangle((-30, -7.5), 60, -1, linewidth=lw, color=color)

    # The paint
    # Create the outer box 0f the paint, width=16ft, height=19ft
    outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=color,
                          fill=False)
    # Create the inner box of the paint, widt=12ft, height=19ft
    inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=color,
                          fill=False)

    # Create free throw top arc
    top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)
    # Create free throw bottom arc
    bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    # Restricted Zone, it is an arc with 4ft radius from center of the hoop
    restricted = Arc((0, 0), 80, 80, theta1=0, theta2=180, linewidth=lw,
                     color=color)

    # Three point line
    # Create the side 3pt lines, they are 14ft long before they begin to arc
    corner_three_a = Rectangle((-220, -47.5), 0, 140, linewidth=lw,
                               color=color)
    corner_three_b = Rectangle((220, -47.5), 0, 140, linewidth=lw, color=color)
    # 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
    # I just played around with the theta values until they lined up with the 
    # threes
    three_arc = Arc((0, 0), 475, 475, theta1=22, theta2=158, linewidth=lw,
                    color=color)

    # Center Court
    center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0,
                           linewidth=lw, color=color)

    # List of the court elements to be plotted onto the axes
    court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                      bottom_free_throw, restricted, corner_three_a,
                      corner_three_b, three_arc, center_outer_arc,
                      center_inner_arc]

    if outer_lines:
        # Draw the half court line, baseline and side out bound lines
        outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)

    # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)

    return ax

### Joint Plot

In [ ]:
# create our jointplot
shot_df=shot_chart_lebron
joint_shot_chart = sns.jointplot(shot_df.LOC_X, shot_df.LOC_Y, stat_func=None,
                                 kind='scatter', space=0, alpha=0.5)

joint_shot_chart.fig.set_size_inches(12,11)

# A joint plot has 3 Axes, the first one called ax_joint 
# is the one we want to draw our court onto and adjust some other settings
ax = joint_shot_chart.ax_joint
draw_court(ax)

# Adjust the axis limits and orientation of the plot in order
# to plot half court, with the hoop by the top of the plot
ax.set_xlim(-250,250)
ax.set_ylim(422.5, -47.5)

# Get rid of axis labels and tick marks
ax.set_xlabel('')
ax.set_ylabel('')
ax.tick_params(labelbottom='off', labelleft='off')

# Add a title
ax.set_title('Lebron James FGA \n2016-17 Reg. Season', 
             y=1.2, fontsize=18)


plt.show()

In [ ]:
import urllib.request
# we pass in the link to the image as the 1st argument
# the 2nd argument tells urlretrieve what we want to scrape
pic = urllib.request.urlretrieve("http://stats.nba.com/media/players/230x185/2544.png",
                                "201935.png")

# urlretrieve returns a tuple with our image as the first 
# element and imread reads in the image as a 
# mutlidimensional numpy array so matplotlib can plot it
lebron_pic = plt.imread(pic[0])

# plot the image
plt.imshow(lebron_pic)
plt.show()

In [ ]:
from matplotlib.offsetbox import  OffsetImage

# create our jointplot

# get our colormap for the main kde plot
# Note we can extract a color from cmap to use for 
# the plots that lie on the side and top axes
cmap=plt.cm.YlOrRd_r 

# n_levels sets the number of contour lines for the main kde plot
joint_shot_chart = sns.jointplot(shot_df.LOC_X, shot_df.LOC_Y, stat_func=None,
                                 kind='kde', space=0, color=cmap(0.1),
                                 cmap=cmap, n_levels=50)

joint_shot_chart.fig.set_size_inches(12,11)

# A joint plot has 3 Axes, the first one called ax_joint 
# is the one we want to draw our court onto and adjust some other settings
ax = joint_shot_chart.ax_joint
draw_court(ax)

# Adjust the axis limits and orientation of the plot in order
# to plot half court, with the hoop by the top of the plot
ax.set_xlim(-250,250)
ax.set_ylim(422.5, -47.5)

# Get rid of axis labels and tick marks
ax.set_xlabel('')
ax.set_ylabel('')
ax.tick_params(labelbottom='off', labelleft='off')

# Add a title
ax.set_title('James Harden FGA \n2014-15 Reg. Season', 
             y=1.2, fontsize=18)


# Add Harden's image to the top right
# First create our OffSetImage by passing in our image
# and set the zoom level to make the image small enough 
# to fit on our plot
img = OffsetImage(lebron_pic, zoom=0.6)
# Pass in a tuple of x,y coordinates to set_offset
# to place the plot where you want, I just played around
# with the values until I found a spot where I wanted
# the image to be
img.set_offset((625,621))
# add the image
ax.add_artist(img)

plt.show()

In [ ]:
shot_chart_lebron

### convert time data

combine minutes remaining and seconds remaining data to produce a single time column with seconds remaining

In [ ]:
shot_chart_lebron['time_remaining_seconds']=shot_chart_lebron['MINUTES_REMAINING']*60+shot_chart_lebron['SECONDS_REMAINING']
data=shot_chart_lebron.drop(['MINUTES_REMAINING','SECONDS_REMAINING'],axis=1)
data

### clutch moments

clucth moments are very important in nba. Especially for one of the greates greatest players of all time (Lebron James). To represent the clucth moments as a feature, we create a new column that shows if the shot is made in the last 10 seconds or not

In [ ]:
data['clutch_shots']=data.time_remaining_seconds.apply(lambda x: 1 if x<=10 else 0)
data

### Home court advantage

In [ ]:
data["Home_court"]=0
for i in range(data.shape[0]):
    if data["TEAM_NAME"].iloc[i]=="Los Angeles Lakers" and data["HTM"].iloc[i]=="LAL":
        data["Home_court"].iloc[i]=1
    elif data["TEAM_NAME"].iloc[i]=="Cleveland Cavaliers" and data["HTM"].iloc[i]=="CLE":
        data["Home_court"].iloc[i]=1
    elif data["TEAM_NAME"].iloc[i]=="Miami Heat	" and data["HTM"].iloc[i]=="MIA":
        data["Home_court"].iloc[i]=1

## Playoff or not

everybody knows that playoff lebron was a different kind of beast. So it is important to include that information in our model

In [ ]:
lebron_playoffs=pd.read_csv('/kaggle/input/lebron_playoffs.csv')["Game_ID"]



In [ ]:
data["playoffs"]=0
data.loc[data["GAME_ID"].isin(lebron_playoffs.values),"playoffs"]=1

## Feature Importance

In [ ]:
data = pd.get_dummies(data.drop(columns=["GAME_ID","EVENT_TYPE",
                                                      "SHOT_ATTEMPTED_FLAG","GAME_DATE","PLAYER_ID","GAME_ID","TEAM_ID",
                                                     "PLAYER_NAME","Unnamed: 0","GRID_TYPE"]))
y =data["SHOT_MADE_FLAG"]
X = data.drop(columns=["SHOT_MADE_FLAG"]) 

In [ ]:
data.shape

## Cart Feature Importance

In [ ]:
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot
model = DecisionTreeClassifier()
model.fit(X, y)
# get importance
importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
cart_importance = dict(zip(X.columns, importance))
{k: v for k, v in sorted(cart_importance.items(), key=lambda item: item[1])}

## Random forest feature importance

In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot
model = RandomForestClassifier()
model.fit(X, y)
importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
randomforest_importance = dict(zip(X.columns, importance))
{k: v for k, v in sorted(randomforest_importance.items(), key=lambda item: item[1])}

## Xgboost feature importance

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X, y)
# get importance
importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print(X.columns[i]+': %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
x_boost_importance = dict(zip(X.columns, importance))
{k: v for k, v in sorted(x_boost_importance.items(), key=lambda item: item[1])}

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
kfold = KFold(n_splits = 5, shuffle = True, random_state = 50)
import lightgbm as lgb
inputs=np.array(X_train)
outputs = np.array(y_train)
x_test = np.array(X_test)

# Empty array for test predictions
y_pred = np.zeros(x_test.shape[0])
    
for train, test in kfold.split(inputs, outputs):
    
    model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                           class_weight = 'balanced', learning_rate = 0.05, 
                           reg_alpha = 0.1, reg_lambda = 0.1, 
                           subsample = 0.8, n_jobs = -1, random_state = 50)
    # Train the model
    model.fit(inputs[train, :], outputs[train], eval_metric = 'auc',
              eval_set = [(inputs[test, :], outputs[test]), (inputs[train, :], outputs[train])],
                  eval_names = ['valid', 'train'],
              early_stopping_rounds = 100, verbose = 200)
    # Record the best iteration
    best_iteration = model.best_iteration_
    print(1)
    # Make predictions
    y_pred += model.predict_proba(x_test, num_iteration = best_iteration)[:, 1] / kfold.n_splits

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
from sklearn.model_selection import GridSearchCV
def gridSearchLgbm(inputs, outputs, X_test, y_test):
    inputs_arr = np.array(inputs)
    outputs_arr = np.array(outputs).reshape(-1, )
    y_test = np.array(y_test).reshape(-1, )
    x_test = np.array(X_test)
    estimator = lgb.LGBMRegressor()
    # grid Search
    gridParams = {
        'learning_rate': [0.05, 0.01],
        'n_estimators': [1000,500],
        'num_leaves': [31, 40,50],
        'boosting_type': ['gbdt'],
        'objective': ['binary'],
        'random_state': [50],  # Updated from 'seed'
        'colsample_bytree': [0.65, 0.7],
        'subsample': [0.6,0.7, 0.8],
        "max_depth": [-1],
         'class_weight' :['balanced']
    }

    gbm = GridSearchCV(estimator, gridParams, cv=3)
    gbm.fit(inputs_arr, outputs_arr, eval_metric='rmse',
            eval_set=[(x_test, y_test)],
            early_stopping_rounds=70, verbose=1000)

    return gbm.best_params_, gbm.best_score_

def lgbm(inputs,outputs,X_test, params=[]):
    kfold = KFold(n_splits = 5, shuffle = True, random_state = 50)
    inputs_arr = np.array(inputs)
    outputs_arr = np.array(outputs).reshape(-1,)
    x_test = np.array(X_test)


    # Empty array for test predictions
    y_pred = np.zeros(x_test.shape[0])

    for train_samp, test_samp in kfold.split(inputs_arr, outputs_arr):
        if len(params)==0:
            model = lgb.LGBMRegressor(n_estimators=10000, objective = 'classification', learning_rate = 0.01,
                            #reg_alpha = 0.7, reg_lambda = 0.5,
                            subsample = 0.8, n_jobs = -1, random_state = 50,
                            importance_type='gain')
        else:
            model = lgb.LGBMRegressor(**params)
      # Train the model
        model.fit(inputs_arr[train_samp, :], outputs_arr[train_samp,], eval_metric = 'rmse',
                    eval_set = [(inputs_arr[test_samp, :], outputs_arr[test_samp,]), (inputs_arr[train_samp, :], outputs_arr[train_samp,])],
                        eval_names = ['valid', 'train'],
                    early_stopping_rounds = 100, verbose = 200)
          # Record the best iteration
        best_iteration = model.best_iteration_
        print(1)
        # Make predictions
        y_pred += model.predict(x_test, num_iteration = best_iteration) / kfold.n_splits
    return y_pred,model.feature_importances_

In [ ]:
best_params, best_score = gridSearchLgbm(inputs, outputs, X_test, y_test)

In [ ]:
best_params


In [ ]:
y_pred, imp2 = lgbm(X_train,y_train,X_test,best_params)


In [ ]:
imp2

In [ ]:
print(roc_auc_score(y_test,y_pred))
print(log_loss(y_test,y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

from collections import OrderedDict

# aim is to optimize 'name_of_parameter'
def optimization_of_parameter_of_rf(X,y,dict_of_param,name_of_parameter, list_of_values, min_estimators, max_estimators):
    list_of_parameter_dicts=[(value,{**dict_of_param,**{'n_estimators':100, 'warm_start':True, 'oob_score':True, 'n_jobs':-1,
                               'random_state':434,name_of_parameter:value}}) for value in list_of_values]
    
    #if x and y are two dictionaries, {**x,**y} is a way of merging these two
    
    ensemble_clfs = [( "{x}={y}".format(x=name_of_parameter, y=value), RandomForestClassifier(**param_dict))
                 for value, param_dict in list_of_parameter_dicts]

    # Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
    error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)



    for label, clf in ensemble_clfs:
        for i in range(min_estimators, max_estimators + 1):
            clf.set_params(n_estimators=i)
            clf.fit(X, y)

            # Record the OOB error for each `n_estimators=i` setting.
            oob_error = 1 - clf.oob_score_
            error_rate[label].append((i, oob_error))

    # Generate the "OOB error rate" vs. "n_estimators" plot.
    plt.figure(figsize=(15,8))
    for label, clf_err in error_rate.items():
        xs, ys = zip(*clf_err)
        plt.plot(xs, ys, label=label)

    plt.xlim(min_estimators, max_estimators)
    plt.xlabel("n_estimators")
    plt.ylabel("OOB error rate")
    plt.legend(loc="upper right")
    plt.show()

In [ ]:
optimization_of_parameter_of_rf(X_train, y_train,{'min_samples_leaf':9},'max_features',[50,70,90],30,250)

In [ ]:
#Evaluation with log loss
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer
#log_scorer=make_scorer(log_loss,greater_is_better=False)
def log_scorer(estimator, X, y):
    pred_probs = estimator.predict_proba(X)[:, 1]
    return log_loss(y, pred_probs)

In [ ]:
from xgboost.sklearn import XGBClassifier
#model = XGBClassifier(colsample_bytree= 0.8, learning_rate= 0.01, max_depth= 7, n_estimators= 400, seed= 1234, subsample= 0.5)
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=400,max_depth=5)
model.fit(X_train, y_train)
pred=model.predict_proba(X_train)[:,1]
log_loss(y_train,pred)
target_y = model.predict_proba(X_test)[:,1]

In [ ]:
print(roc_auc_score(y_train,pred))
print(log_loss(y_train,pred))